In [85]:
import cv2
import csv
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

In [86]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases


class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

class Loss:
    def calculate(self, y_pred, y):
        sample_losses = self.forward(y_pred, y)
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]

        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped*y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods


#MAIN HERE

LOAD DATA

In [87]:
df = pd.read_csv('dataset.csv')
csv = df.to_numpy()
np.random.shuffle(csv)
X = csv[:,0:-1]
Y = csv[:,0]
DATA_VOLUME,PIXELS = X.shape
PIXELS

784

In [88]:
# Initialize Layers and Activation
hidden_layer = Layer_Dense(PIXELS,100)
activator = Activation_ReLU()
output_layer = Layer_Dense(100,10)
output_softmax = Activation_Softmax()
loss_funtion = Loss_CategoricalCrossentropy()


TRAIN DATA

In [90]:

# Run the model
# for sample in X:
hidden_layer.forward(X)
activator.forward(hidden_layer.output)
output_layer.forward(activator.output)
output_softmax.forward(output_layer.output)
loss = loss_funtion.calculate(output_softmax.output,Y)

print(Y[:5])
print(output_layer.output[:5])

# output_layer = Layer_Dense(hidden_layer.output,10)

[9 8 3 7 1]
[[  51.0570603    68.96913208 -111.66145695  -11.60518953  -82.43795935
   -69.11828309  -22.43863128  -85.88212768  -39.18058183   -5.43677899]
 [  76.73792556   56.55615295  -24.11692287  -92.25662636  -30.74709057
   -21.51225721   28.36517662  -42.88283896  -88.98667093  -27.90256323]
 [  84.06656421   29.96596067  -42.70620015  -68.3750593   -15.52622989
   -26.55088156    7.96731052  -35.69427824  -23.70228343  -11.64960015]
 [  70.77294507   58.44659125  -84.55889361  -34.95454831  -47.61284623
   -38.79361061   -7.03150782  -74.02895087    6.70570195   14.90316366]
 [  59.98516066   52.28709843  -70.31391508  -24.71504106  -45.37902027
   -38.0538325   -12.45200523  -66.57713891   -1.49909617   33.93609821]]
